# Imports

###### Here I will be importing the necessary packages to use for this project. Some may not be used but could be used and I am covering my baises with them incase I do use one later on.

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import seaborn as sns
import math

from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
import scipy.stats as stats
from sklearn.feature_selection import RFE


from IPython.display import Image
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import string
from nltk.corpus import stopwords
from nltk import word_tokenize
import scipy

import types
import re
import nltk
import xgboost

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, GridSearchCV

import sys

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
import pickle

from keras.preprocessing import text, sequence
import warnings
warnings.filterwarnings('ignore')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/elliott/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/elliott/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Reading in the Data

###### Data cleaned by partner 

In [2]:
df = pd.read_csv("../../../data/thanos.csv")
df.head(3)

,steamid,appid,app_title,app_tags,review,fps,voted_up
0,76561198271088129,4000,Garry's Mod,"['Sandbox', 'Multiplayer', 'Funny', 'Moddable'...",good models\n,True,True
1,76561198138206834,4000,Garry's Mod,"['Sandbox', 'Multiplayer', 'Funny', 'Moddable'...",I completely suck at making anything on here a...,True,True
2,76561198128760839,70,Half-Life,"['FPS', 'Sci-fi', 'Action', 'Singleplayer', ""1...",There is not much to say about this old game t...,True,True


# Functions for use

###### Functions create by teammate for use on models and to prepare the data for modeling.

In [3]:
def ScoreModel(model, X, y):
    preds = model.predict(X)
    acc = accuracy_score(y, preds)
    f1 = f1_score(y, preds)
    recall = recall_score(y, preds)
    precision = precision_score(y, preds)
    rockout = roc_auc_score(y, preds)
    
    print("Accuracy:  ", acc)
    print("F1 Score:  ", f1)
    print("Recall:    ", recall)
    print("Precision: ", precision)
    print("ROC_AUC:   ", rockout)

In [4]:
def CleanText(reviews):
    stopwords = nltk.corpus.stopwords.words('english')
    punct = string.punctuation
    lemma = nltk.WordNetLemmatizer()
    
    reviews = "".join([word for word in reviews if word not in string.punctuation])
    tokens = re.split('\W+', reviews)
    reviews = [lemma.lemmatize(word) for word in tokens if word not in stopwords]
    
    
    return reviews


In [5]:
def clean_reviews(reviews):
    reviews = "".join([word for word in reviews if word not in string.punctuation])
    tokens = re.split('\W+', reviews)
    reviews = [lemma.lemmatize(word) for word in tokens if word not in stopwords]
    return reviews

###### Checking on the data with a .info() method in order to check the data for nulls 

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23183 entries, 0 to 23182
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   steamid    23183 non-null  int64 
 1   appid      23183 non-null  int64 
 2   app_title  23183 non-null  object
 3   app_tags   23183 non-null  object
 4   review     23183 non-null  object
 5   fps        23183 non-null  bool  
 6   voted_up   23183 non-null  bool  
dtypes: bool(2), int64(2), object(3)
memory usage: 951.0+ KB


###### Looking at the percentage of the column voted_up based on value count.

In [7]:
df["voted_up"].value_counts(normalize=True)

True     0.505888
False    0.494112
Name: voted_up, dtype: float64

###### Using a function above to creat a new column in the data frame for use later int he notebook.

In [8]:
df['clean_review'] = df['review'].apply(lambda x: CleanText(x.lower()))

df.head()

,steamid,appid,app_title,app_tags,review,fps,voted_up,clean_review
0,76561198271088129,4000,Garry's Mod,"['Sandbox', 'Multiplayer', 'Funny', 'Moddable'...",good models\n,True,True,"[good, model, ]"
1,76561198138206834,4000,Garry's Mod,"['Sandbox', 'Multiplayer', 'Funny', 'Moddable'...",I completely suck at making anything on here a...,True,True,"[completely, suck, making, anything, death, ru..."
2,76561198128760839,70,Half-Life,"['FPS', 'Sci-fi', 'Action', 'Singleplayer', ""1...",There is not much to say about this old game t...,True,True,"[much, say, old, game, hasnt, said, really, ma..."
3,76561198079636858,6060,"Star Wars: Battlefront 2 (Classic, 2005)","['Action', 'Multiplayer', 'Shooter', 'Third-Pe...",360 noscoped almost everything 10/10 even the ...,True,True,"[360, noscoped, almost, everything, 1010, even..."
4,76561198238100200,220,Half-Life 2,"['FPS', 'Action', 'Sci-fi', 'Classic', 'Single...",No need for a review. It's practically history.,True,True,"[need, review, practically, history]"


# Start of the preprocessing 
Vectorizing and train test splitting the data

In [12]:
%%timeit 
vector = TfidfVectorizer(analyzer=CleanText, ngram_range=(2, 2))
X = vector.fit_transform(df["review"])

X_df = pd.DataFrame(X.toarray())
X_df.columns = vector.get_feature_names()
X_df.head(5)

11.4 s ± 250 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
vector_pikl=vector.fit(df['review'])
vector_pikl

TfidfVectorizer(analyzer=<function CleanText at 0x7f8271a90790>,
                ngram_range=(2, 2))

In [10]:
y = df[["voted_up"]]

X_train, X_test, y_train, y_test = train_test_split(X_df, y, random_state=57, test_size = 0.5,stratify=y)

# Baseline model

Using a simple logistic regression model as the baseling model.

In [11]:
baseline = LogisticRegression()
baseline.fit(X_train, y_train)
ScoreModel(baseline, X_train, y_train)

Accuracy:   0.912173237856958
F1 Score:   0.9129765771926825
Recall:     0.9106412005457026
Precision:  0.9153239629756599
ROC_AUC:    0.9121915623821581


In [12]:
ScoreModel(baseline, X_test, y_test)

Accuracy:   0.814095928226363
F1 Score:   0.8169229462237703
Recall:     0.819918144611187
Precision:  0.813949551379719
ROC_AUC:    0.814026809735761


# Baseline results 

Results where decent not great so we try a MultinomialNB model now to check the score diffrences. 

# MultinomialNB Model 


In [13]:
clf = MultinomialNB()
clf.fit(X_train, y_train)
ScoreModel(clf, X_train, y_train)

Accuracy:   0.90156155637995
F1 Score:   0.8981704596162428
Recall:     0.8581173260572987
Precision:  0.9421456656056918
ROC_AUC:    0.9020811879107866


In [14]:
ScoreModel(clf, X_test, y_test)

Accuracy:   0.7979641131815045
F1 Score:   0.7863918278000729
Recall:     0.7351637107776262
Precision:  0.8452941176470589
ROC_AUC:    0.7987096486849025


# Results

Results where better than the baseline we will now try one more Multinomial model with a one parameter grid search to test the scores.

# Starting a Grid Search MultinomialNB Model

In [15]:
param_grid = {'alpha': [.05,0.5, 1, 1.5, 2],
        }

In [16]:
clf2 = MultinomialNB()
clf2.fit(X_train, y_train)


clf_gs = GridSearchCV(estimator=clf2, param_grid=param_grid, scoring='accuracy')
clf_gs.fit(X_train, y_train)


GridSearchCV(estimator=MultinomialNB(),
             param_grid={'alpha': [0.05, 0.5, 1, 1.5, 2]}, scoring='accuracy')

In [17]:

best_model = clf_gs.best_estimator_

In [18]:

clf_gs.best_params_

{'alpha': 0.5}

In [19]:
ScoreModel(clf_gs, X_train, y_train)

Accuracy:   0.9197653351738417
F1 Score:   0.9184067380242147
Recall:     0.8925648021828103
Precision:  0.9457896638959161
ROC_AUC:    0.9200906776760045


In [20]:
ScoreModel(clf_gs, X_test, y_test)

Accuracy:   0.8019323671497585
F1 Score:   0.794522999821013
Recall:     0.7569918144611187
Precision:  0.835969868173258
ROC_AUC:    0.8024658792975985


# Results of Grid Search
The results were similar but took substanially longer with no true diffrence in scores. Causing us to choose the first MultinomialNB model as our final model. Below we pickled our model in order to deploy the model on a application along with the reccomendation side of the project.  

In [27]:
with open('nlp_model.pkl', 'wb') as f:
    pickle.dump(clf, f)

In [28]:
with open('vector_fit.pkl', 'wb') as f:
    pickle.dump(vector_pikl, f)